In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def calibrate_camera_from_folder(folder_path, chessboard_size, square_size):
    ''' Calibrate camera using images of a chessboard pattern taken from a folder.
    Args: 
        folder_path: Path to the folder containing images of a chessboard pattern
        chessboard_size: Tuple of INNER corners in the chessboard pattern, e.g., (11, 7) for a 12x8 chessboard pattern
        square_size: Size of a square in the chessboard pattern in millimeters
    '''
    objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    # generate coordinates of chessboard corners with mgrid
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    # multiply by square_size to get the actual real-world metric coordinates
    objp *= square_size

    objpoints = []  # 3D points in real-world space, e.g., (0,0,0), (1,0,0), (2,0,0) ....
    imgpoints = []  # 2D points in image plane - inner corners of the chessboard

    images = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if fname.endswith(('.jpg', '.png', '.jpeg'))]

    for image_path in images:
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        if ret:
            objpoints.append(objp) # Add the 3D real-world coordinates
            imgpoints.append(corners) # Add the corresponding 2D image coordinates

            # Draw and display the corners
            cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
            plt.imshow(img)
            plt.axis('off')  # Turn off axes for cleaner display
            plt.show()

    # Calibrate the camera - Zhang's method
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    if ret:
        print("Camera calibration successful!")
        print("Camera matrix:")
        print(camera_matrix)
        print("Distortion coefficients:")
        print(dist_coeffs)
    else:
        print("Camera calibration failed.")

    return ret, camera_matrix, dist_coeffs, rvecs, tvecs

folder_path = "chessboard"
chessboard_size = (11, 7)  # Adjust based on your chessboard pattern (inner corners)
square_size = 40.0  # Size of a square in millimeters

# Run calibration
ret, camera_matrix, dist_coeffs, rvecs, tvecs = calibrate_camera_from_folder(folder_path, chessboard_size, square_size)


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Load two images for initial registration
folder_path = "fountain_dataset"
# Load all images in the folder and sort them
image_paths = sorted(
    [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if fname.endswith(('.jpg', '.png', '.jpeg'))]
)
image1 = cv2.imread(image_paths[0]) 
image2 = cv2.imread(image_paths[1])

# Check if images are loaded successfully
if image1 is None:
    print("Error: Could not load image")
    exit()

plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Turn off axes for cleaner display
plt.show()

if image2 is None:
    print("Error: Could not load image")
    exit()

plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Turn off axes for cleaner display
plt.show()


In [ ]:
def detect_and_draw_features(image, detector_type='SIFT'):
    ''' Detect features in an image using a feature detector.
    Args:
        image: The input image
        detector_type: Type of feature detector to use. Default is 'SIFT'. Other options are 'ORB' and 'AKAZE'.
    '''
    if image is None:
        print(f"Error: baaad inmage")
        return
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize the feature detector
    if detector_type == 'SIFT':
        detector = cv2.SIFT_create()
    elif detector_type == 'ORB':
        detector = cv2.ORB_create()
    elif detector_type == 'AKAZE':
        detector = cv2.AKAZE_create()
    else:
        print(f"Error: Unsupported detector type '{detector_type}'. Use 'SIFT', 'ORB', or 'AKAZE'.")
        return

    # Detect features and compute descriptors
    keypoints, descriptors = detector.detectAndCompute(gray, None)

    # Draw the keypoints on the image
    output_image = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Convert BGR to RGB for displaying with matplotlib
    output_image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

    # Display the result
    plt.figure(figsize=(10, 10))
    plt.imshow(output_image_rgb)
    plt.title(f"Feature Detection with {detector_type}")
    plt.axis('off')
    plt.show()

    print(f"Detected {len(keypoints)} features using {detector_type}.")

    return keypoints, descriptors
    
# Detect features and descriptors in the two images, make sure to use the same detector for both images
keys1, descs1 = detect_and_draw_features(image1, detector_type='SIFT')
keys2, descs2 = detect_and_draw_features(image2, detector_type='SIFT')

In [ ]:
def match_and_draw_features(image1, image2, keypoints1, descriptors1, keypoints2, descriptors2, matcher_type='BF', metric='L2'):
    ''' Match features between two images using a feature matcher.
    Args:
        image1: The first input image
        image2: The second input image
        keypoints1: Keypoints detected in the first image
        descriptors1: Descriptors computed for the keypoints in the first image
        keypoints2: Keypoints detected in the second image
        descriptors2: Descriptors computed for the keypoints in the second image
        matcher_type: Type of feature matcher to use. Default is 'BF'. Other option is 'FLANN'.
        metric: Distance metric to use for matching. Default is 'L2'. Other option is 'L1' for Manhattan distance
    '''
    if matcher_type == 'BF':
        # Brute-force matcher
        matcher = cv2.BFMatcher(metric if metric == 'L1' else cv2.NORM_L2, crossCheck=True)
    elif matcher_type == 'FLANN':
        # FLANN-based matcher
        index_params = dict(algorithm=1, trees=5)  # Algorithm 1 = KDTree
        search_params = dict(checks=50)  # Number of checks
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
    else:
        print(f"Error: Unsupported matcher type '{matcher_type}'. Use 'BF' or 'FLANN'.")
        return

    # Match descriptors
    matches = matcher.match(descriptors1, descriptors2)

    # Draw matches
    matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[::5], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Convert BGR to RGB for displaying with matplotlib
    matched_image_rgb = cv2.cvtColor(matched_image, cv2.COLOR_BGR2RGB)

    # Display the matched image
    plt.figure(figsize=(15, 10))
    plt.imshow(matched_image_rgb)
    plt.title(f"Feature Matches ({matcher_type})")
    plt.axis('off')
    plt.show()

    print(f"Number of matches: {len(matches)}")

    return matches

# match features in the two images
matches = match_and_draw_features(image1, image2, keys1, descs1, keys2, descs2, matcher_type='BF')

In [ ]:
def compute_projection_matrices_and_filter_matches(matches, keypoints1, keypoints2, K):
    ''' Compute the projection matrices of the two cameras and filter matches using the essential matrix.
    Args:
        matches: List of matches between the two images
        keypoints1: Keypoints detected in the first image
        keypoints2: Keypoints detected in the second image
        K: Camera intrinsics matrix
    '''

    # Extract matched keypoints
    pts1 = np.array([keypoints1[m.queryIdx].pt for m in matches], dtype=np.float32)
    pts2 = np.array([keypoints2[m.trainIdx].pt for m in matches], dtype=np.float32)

    # we have 2 options
    # either 1. compute Fundamental matrix and using K, compute Essential matrix
    # or 2. directly compute Essential matrix
    
    # option 1
    #F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, 1.0, 0.999)
    #E = K.T @ F @ K

    # option 2
    E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)

    # Filter matches using the mask
    inlier_matches = [m for m, inlier in zip(matches, mask.ravel()) if inlier]
    pts1_inliers = pts1[mask.ravel() == 1]
    pts2_inliers = pts2[mask.ravel() == 1]

    # Recover pose (R and T) from the essential matrix
    _, R, T, mask_pose = cv2.recoverPose(E, pts1_inliers, pts2_inliers, K)

    # Form the projection matrices
    P1 = np.dot(K, np.hstack((np.eye(3), np.zeros((3, 1)))))  # First camera at origin of world coordinates
    P2 = np.dot(K, np.hstack((R, T)))  # Second camera with R, T

    print("Essential Matrix (E):")
    print(E)
    print("\nRotation (R):")
    print(R)
    print("\nTranslation (T):")
    print(T)
    print(f"\nNumber of inlier matches: {len(inlier_matches)}")
    print("\nProjection Matrix of Camera 1 (P1):")
    print(P1)
    print("\nProjection Matrix of Camera 2 (P2):")
    print(P2)

    return P1, P2, R, T, inlier_matches, pts1_inliers, pts2_inliers, mask.ravel()

# Visualize the inlier matches (optional)
def visualize_inlier_matches(image1, image2, keypoints1, keypoints2, inlier_matches):
    matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, inlier_matches[::10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    matched_image_rgb = cv2.cvtColor(matched_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(15, 10))
    plt.imshow(matched_image_rgb)
    plt.title(f"Inlier Matches ({len(inlier_matches)} matches)")
    plt.axis('off')
    plt.show()

# Intrinsics matrix (K) for fountain dataset
# It is different camera from the one used in the chessboard calibration
# todo: make consistent
K = np.array([
    [689.870116, -0.001629, 380.172501],  # fx, 0, cx
    [0, 691.039531, 251.702243],  # 0, fy, cy
    [0, 0, 1]       # 0, 0, 1
], dtype=np.float64)

# Compute projection matrices and filter matches
P1, P2, R, T, inlier_matches, pts1_inliers, pts2_inliers, mask = compute_projection_matrices_and_filter_matches(matches, keys1, keys2, K)

# camera poses - we will keep adding the camera poses as we move along
camera_poses = [np.eye(4)]  # first camera is at the origin
camera_poses.append(np.hstack((R, T))) # second camera pose

# Visualize inlier matches
visualize_inlier_matches(image1, image2, keys1, keys2, inlier_matches);

In [ ]:
import plotly.graph_objects as go

def triangulate_3d_points(P1, P2, pts1, pts2):
    ''' Triangulate 3D points from corresponding 2D points in two images.
    Args:
        P1, P2: Projection matrices of camera 1 and camera 2
        pts1, pts2: Corresponding 2D points in the two images
    '''

    # Ensure points are in the correct shape (2xN for triangulation)
    pts1_homogeneous = pts1.T
    pts2_homogeneous = pts2.T

    # Perform triangulation (returns 4D homogeneous coordinates)
    points_4d_homogeneous = cv2.triangulatePoints(P1, P2, pts1_homogeneous, pts2_homogeneous)

    # Convert homogeneous coordinates to 3D (divide by w)
    points_3d = points_4d_homogeneous[:3] / points_4d_homogeneous[3]

    # Transpose to return points in Nx3 format
    return points_3d.T

def update_structure_and_map(structure, keypoint_to_structure_map, keys1, keys2, inlier_matches, image1_id, image2_id, points_3d):
    ''' Update the 3D structure and keypoint-to-structure map using the new 3D points.
    Args:
        structure: The current 3D structure
        keypoint_to_structure_map: Mapping of keypoints to 3D points
        keys1, keys2: Keypoints detected in the two images
        inlier_matches: List of inlier matches between the two images
        image1_id, image2_id: IDs of the two images
        points_3d: new triangulated 3D points
    '''
    # Add 3D points to the structure and update the map
    for i, match in enumerate(inlier_matches):
        # key will be touple [image_id, keypoint_id]
        kp_id1 = (image1_id, match.queryIdx)
        kp_id2 = (image2_id, match.trainIdx)

        # Add the 3D point to the structure
        point_idx = len(structure)  # Index of the new point
        structure.append(points_3d[i])  # Add point to structure

        # Map keypoints to the new 3D point index
        keypoint_to_structure_map[kp_id1] = point_idx
        keypoint_to_structure_map[kp_id2] = point_idx

def plot_3d_points_interactive(points_3d, camera_poses, x_range=(-5, 5), y_range=(-5, 5), z_range=(0, 10)):
    ''' Plot 3D points and camera centers interactively using Plotly.
    Args:
        points_3d: 3D points to plot (Nx3 array).
        camera_poses: List of [R|t] matrices (3x4 arrays) for camera extrinsics.
        x_range, y_range, z_range: Ranges for the X, Y, Z axes.
    '''
    # Crop points to the specified ranges
    mask = (
        (points_3d[:, 0] >= x_range[0]) & (points_3d[:, 0] <= x_range[1]) &  # x range
        (points_3d[:, 1] >= y_range[0]) & (points_3d[:, 1] <= y_range[1]) &  # y range
        (points_3d[:, 2] >= z_range[0]) & (points_3d[:, 2] <= z_range[1])    # z range
    )
    cropped_points = points_3d[mask]

    # Extract camera centers from [R|t] matrices
    camera_centers = []
    for pose in camera_poses:
        R = pose[:, :3]  # Rotation matrix (3x3)
        t = pose[:, 3]   # Translation vector (3x1)
        C = -np.dot(R.T, t)  # Camera center in world coordinates
        camera_centers.append(C)
    camera_centers = np.array(camera_centers)  # Convert to Nx3 array

    # Create a 3D scatter plot
    fig = go.Figure()

    # Add 3D points
    fig.add_trace(go.Scatter3d(
        x=cropped_points[:, 0],  # X-coordinates
        y=cropped_points[:, 2],  # Y-coordinates
        z=-cropped_points[:, 1],  # Z-coordinates
        mode='markers',
        marker=dict(size=3, color=cropped_points[:, 2], colorscale='Viridis', opacity=0.8),
        name="3D Points"
    ))

    # Add camera centers
    fig.add_trace(go.Scatter3d(
        x=camera_centers[:, 0],  # X-coordinates of cameras
        y=camera_centers[:, 2],  # Y-coordinates of cameras
        z=-camera_centers[:, 1],  # Z-coordinates of cameras
        mode='markers',
        marker=dict(size=5, color='red', symbol='x'),
        name="Camera Centers"
    ))

    # Set plot layout
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        title="Interactive 3D Point Cloud with Camera Centers",
    )
    fig.show()

# Perform triangulation
points_3d = triangulate_3d_points(P1, P2, pts1_inliers, pts2_inliers)

# we will keep track of the 3D points and the mapping of keypoints to 2D points
# structure will store the 3D points ->
structure = []
# keypoint_to_structure_map will store the mapping of keypoints to 3D points
keypoint_to_structure_map = {}

# store the 3D points and update the map
update_structure_and_map(structure, keypoint_to_structure_map, keys1, keys2, inlier_matches, 0, 1, points_3d)

# Print the 3D points
print("Triangulated {points_3d.shape[0]} 3D points:")

# Plot the triangulated points interactively
plot_3d_points_interactive(points_3d, camera_poses)

In [ ]:
# here we will add a new image to the structure

def find_2d_3d_correspondences(matches, keypoints_new, image_id_past):
    ''' Find 2D-3D correspondences between two images using the matches and the keypoint-to-structure map.
    Args:
        matches: List of matches between the two images
        keypoints1, keypoints2: Keypoints detected in the two images
        image1_id, image2_id: IDs of the two images
    '''
    pts_2d = []
    pts_3d = []

    for match in matches:
        # look for the keypoint in the previous image
        kp_id = (image_id_past, match.queryIdx)

        # Check if the keypoint in the previous image is mapped to a 3D point
        if kp_id in keypoint_to_structure_map:
            # Retrieve the 3D point index
            point_3d_idx = keypoint_to_structure_map[kp_id]

            # Get the 2D point from the current image (trainIdx)
            pts_2d.append(keypoints_new[match.trainIdx].pt)

            # Retrieve the corresponding 3D point from the structure
            pts_3d.append(structure[point_3d_idx])

    return np.array(pts_2d, dtype=np.float32), np.array(pts_3d, dtype=np.float32)

def compute_pose_p3p(K, pts_3d, pts_2d):
    ''' Compute the camera pose using the Perspective-n-Point (P3P) algorithm.
    Args:
        K: Camera intrinsics matrix
        pts_3d: 3D points in world coordinates
        pts_2d: 2D points in image coordinates
    '''
    success, rvec, tvec, inliers = cv2.solvePnPRansac(
        pts_3d, pts_2d, K, None, flags=cv2.SOLVEPNP_P3P
    )

    if not success:
        raise ValueError("P3P failed to compute a valid pose.")

    # Convert rotation vector to a rotation matrix (opencv represents rotation as a vector)
    R, _ = cv2.Rodrigues(rvec)

    return R, tvec, inliers

def filter_matches_using_epipolar_constraint(matches, keypoints1, keypoints2, threshold=0.1):
    ''' Filter matches using the epipolar constraint.
    Args:
        matches: List of matches between the two images
        keypoints1, keypoints2: Keypoints detected in the two images
        threshold: Threshold for the epipolar error
    '''
    # Extract matched keypoints
    pts1 = np.array([keypoints1[m.queryIdx].pt for m in matches], dtype=np.float32)
    pts2 = np.array([keypoints2[m.trainIdx].pt for m in matches], dtype=np.float32)

    # Convert keypoints to homogeneous coordinates
    pts1_homogeneous = np.hstack((pts1, np.ones((pts1.shape[0], 1))))
    pts2_homogeneous = np.hstack((pts2, np.ones((pts2.shape[0], 1))))

    # Compute the fundamental matrix
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, threshold, 0.999)

    # Filter matches using the mask
    inlier_matches = [m for m, inlier in zip(matches, mask.ravel()) if inlier]

    print(f"Number of matches before filtering: {len(matches)}")
    print(f"Number of matches after filtering: {len(inlier_matches)}")

    return inlier_matches

def compute_projection_matrix(K, R, T):
    ''' Compute the projection matrix from the camera pose.
    Args:
        K: Camera intrinsics matrix
        R: Rotation matrix
        T: Translation vector
    '''
    P = np.dot(K, np.hstack((R, T)))
    return P

def triangulate_3d_points_matches(P1, P2, kpts1, kpts2, matches):
    ''' Triangulate 3D points from matched keypoints in two images.
    Args:
        P1, P2: Projection matrices of camera 1 and camera 2
        kpts1, kpts2: Keypoints detected in the two images
        matches: List of matches between the two images
    '''
    # Extract matched keypoints
    pts1 = np.array([kpts1[m.queryIdx].pt for m in matches], dtype=np.float32).T  # Shape: 2xN
    pts2 = np.array([kpts2[m.trainIdx].pt for m in matches], dtype=np.float32).T  # Shape: 2xN

    # Perform triangulation using OpenCV
    points_4d_homogeneous = cv2.triangulatePoints(P1, P2, pts1, pts2)

    # Convert homogeneous coordinates to 3D (divide by w)
    points_3d = points_4d_homogeneous[:3] / points_4d_homogeneous[3]

    # Transpose to return points in Nx3 format
    return points_3d.T

def add_new_image(path, past_id, past_image, past_P, past_keys, past_descs):
    # load next image
    image_new = cv2.imread(path)

    # extract keypoints
    keys_new, descs_new = detect_and_draw_features(image_new, detector_type='SIFT')

    # match to previous image
    matches_new = match_and_draw_features(past_image, image_new, past_keys, past_descs, keys_new, descs_new, matcher_type='BF')

    # compute camera position from p3p
    pts_2d, pts_3d = find_2d_3d_correspondences(matches_new, keys_new, past_id)
    print("Found ",pts_3d.shape[0]," 2D-3D point correspondences.")

    # Compute camera pose for the new image using P3P algorithm
    R, T, inliers = compute_pose_p3p(K, pts_3d, pts_2d)
    camera_poses.append(np.hstack((R, T))) # add the new camera pose

    # Update the projection matrix for the new image
    P_new = compute_projection_matrix(K, R, T)

    # filter matches using epipolar constraint and projection matrices
    matches_new = filter_matches_using_epipolar_constraint(matches_new, past_keys, keys_new)
    
    print("Projection Matrix for Camera ",past_id+2," (P_new):")
    print(P_new)

    # Triangulate 3D points using the new projection matrix
    new_points_3d = triangulate_3d_points_matches(past_P, P_new, past_keys, keys_new, matches_new)

    # Update the 3D structure and keypoint-to-structure map
    update_structure_and_map(structure, keypoint_to_structure_map, past_keys, keys_new, matches_new, past_id, past_id + 1, new_points_3d)

    return past_id+1, image_new, P_new, keys_new, descs_new, new_points_3d

# we will define 'past' image data (image 2) to automatically add new images
last_id = 1
last_keys = keys2
last_descs = descs2
last_image = image2
last_P = P2

last_id, last_image, last_P, last_keys, last_descs, new_points_3d = add_new_image(image_paths[last_id + 1] , last_id, last_image, last_P, last_keys, last_descs)
points_3d = np.vstack((points_3d, new_points_3d))
plot_3d_points_interactive(points_3d, camera_poses)

In [ ]:
last_id, last_image, last_P, last_keys, last_descs, new_points_3d = add_new_image(image_paths[last_id + 1] , last_id, last_image, last_P, last_keys, last_descs)
points_3d = np.vstack((points_3d, new_points_3d))
plot_3d_points_interactive(points_3d, camera_poses)

In [ ]:
last_id, last_image, last_P, last_keys, last_descs, new_points_3d = add_new_image(image_paths[last_id + 1] , last_id, last_image, last_P, last_keys, last_descs)
points_3d = np.vstack((points_3d, new_points_3d))
plot_3d_points_interactive(points_3d, camera_poses)

In [ ]:
last_id, last_image, last_P, last_keys, last_descs, new_points_3d = add_new_image(image_paths[last_id + 1] , last_id, last_image, last_P, last_keys, last_descs)
plot_3d_points_interactive(points_3d, camera_poses)

In [ ]:
last_id, last_image, last_P, last_keys, last_descs, new_points_3d = add_new_image(image_paths[last_id + 1] , last_id, last_image, last_P, last_keys, last_descs)
points_3d = np.vstack((points_3d, new_points_3d))
plot_3d_points_interactive(points_3d, camera_poses)

In [ ]:
# you can try adding rest of the images in the dataset
# self todo: optimization (refinement)